In [1]:
import numpy as np
from ompl import base as ob
from ompl import geometric as og
from pydrake.planning import RobotDiagramBuilder
from pydrake.math import RigidTransform
from airo_drake import add_meshcat, finish_build, visualize_frame
import airo_models
from dataclasses import dataclass
from ConstrainedPlanningCommon import ConstrainedProblem


Drake failed to load a required library. This could indicate an installation
problem, or that your system is missing required distro-provided packages.
Please refer to the installation instructions to ensure that all required
dependencies are installed.

For Ubuntu 22.04, install these additional libraries:

    sudo apt-get install --no-install-recommends \
      libx11-6 libsm6 libglib2.0-0

For Ubuntu 24.04, install these additional libraries:

    sudo apt-get install --no-install-recommends \
      libx11-6 libsm6 libglib2.0-0t64

For macOS, ensure that you're using Homebrew Python (not Apple's system Python).

For more information, please see https://drake.mit.edu/installation.html



ImportError: libX11.so.6: cannot open shared object file: No such file or directory

In [ ]:
# Create the robot diagram builder
robot_diagram_builder = RobotDiagramBuilder()

# Add meshcat visualizer
meshcat = add_meshcat(robot_diagram_builder)

# Get plant and parser from the builder
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

# Create two cubes using airo_models
cube1_size = (0.2, 0.2, 0.2)
cube2_size = (0.2, 0.2, 0.2)
cube1_urdf_path = airo_models.box_urdf_path(cube1_size, "cube1")
cube2_urdf_path = airo_models.box_urdf_path(cube2_size, "cube2")

# Add cubes to the scene
cube1_index = parser.AddModels(cube1_urdf_path)[0]
cube2_index = parser.AddModels(cube2_urdf_path)[0]

# Position the cubes in space
world_frame = plant.world_frame()
cube1_frame = plant.GetFrameByName("base_link", cube1_index)
cube2_frame = plant.GetFrameByName("base_link", cube2_index)

# Set transforms for the cubes
cube1_transform = RigidTransform(p=[0.5, 0.0, 0.3])  # Position cube1 at (0.5, 0, 0.3)
cube2_transform = RigidTransform(p=[-0.5, 0.0, 0.3])  # Position cube2 at (-0.5, 0, 0.6)

# Weld the cubes to the world frame at their positions
plant.WeldFrames(world_frame, cube1_frame, cube1_transform)
plant.WeldFrames(world_frame, cube2_frame, cube2_transform)

# Finish building the scene
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

# Force publish to visualize
robot_diagram.ForcedPublish(context)

# Visualize the frames of the cubes
visualize_frame(meshcat, "cube1_frame", cube1_transform, length=0.25)
visualize_frame(meshcat, "cube2_frame", cube2_transform, length=0.25)

In [3]:
# Define goal state
goal_transform1 = RigidTransform(p=[0.3, 0.4, 0.5])
goal_transform2 = RigidTransform(p=[-0.3, 0.4, 0.5])

In [4]:
# Setup simple motion planning
space = ob.RealVectorStateSpace(6)  # 6D state space (3D + 3D)

# Set bounds for the state space
bounds = ob.RealVectorBounds(6)
for i in range(6):
    bounds.setLow(i, -2.0)   # Increased bounds for more freedom
    bounds.setHigh(i, 2.0)
space.setBounds(bounds)

# Create simple setup
ss = og.SimpleSetup(space)

In [ ]:
# Set the start state
start = ob.State(space)
start()[0] = cube1_transform.translation()[0]  # x1
start()[1] = cube1_transform.translation()[1]  # y1
start()[2] = cube1_transform.translation()[2]  # z1
start()[3] = cube2_transform.translation()[0]  # x2
start()[4] = cube2_transform.translation()[1]  # y2
start()[5] = cube2_transform.translation()[2]  # z2

# Set the goal state
goal = ob.State(space)
goal()[0] = goal_transform1.translation()[0]  # x1
goal()[1] = goal_transform1.translation()[1]  # y1
goal()[2] = goal_transform1.translation()[2]  # z1
goal()[3] = goal_transform2.translation()[0]  # x2
goal()[4] = goal_transform2.translation()[1]  # y2
goal()[5] = goal_transform2.translation()[2]  # z2

# Set the start and goal states
ss.setStartAndGoalStates(start, goal)

# Set the planner (RRTConnect tends to be good for simple problems)
planner = og.RRTConnect(ss.getSpaceInformation())
ss.setPlanner(planner)

# Try to solve the problem
solved = ss.solve(5.0)  # 5 seconds planning time

if solved:
    print("Found solution!")
    # Simplify & interpolate the solution
    ss.simplifySolution()
    path = ss.getSolutionPath()
    path.interpolate(50)  # Create a smoother path with more waypoints
    
    # Visualize the solution
    print(f"Path has {path.getStateCount()} states")
    for i in range(path.getStateCount()):
        state = path.getState(i)
        
        # Create transforms for visualization
        cube1_pos = [state[0], state[1], state[2]]
        cube2_pos = [state[3], state[4], state[5]]
        
        cube1_transform_i = RigidTransform(p=cube1_pos)
        cube2_transform_i = RigidTransform(p=cube2_pos)
        
        # Visualize frames
        visualize_frame(meshcat, f"cube1_frame_{i}", cube1_transform_i, length=0.15)
        visualize_frame(meshcat, f"cube2_frame_{i}", cube2_transform_i, length=0.15)
        
        # Add small delay for visualization
        import time
        time.sleep(0.1)
        
    # Visualize goal frames
    visualize_frame(meshcat, "cube1_frame_goal", goal_transform1, length=0.15)
    visualize_frame(meshcat, "cube2_frame_goal", goal_transform2, length=0.15)
else:
    print("No solution found")